In [8]:
import pandas as pd
import numpy as np

from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

import matplotlib.pyplot as plt

import glob, os
import seaborn as sns
import sys
# from sklearn.preprocessing import MinMaxScale


In [9]:
'''
** 训练数据有8列：**

- 日期 - 年: int
- 日期 - 月: int
- 日期 - 日: int， 时间跨度为2015年2月1日 - 2016年8月31日
- 当日最高气温 - 摄氏度（下同）: float
- 当日最低气温: float
- 当日平均气温: float
- 当日平均湿度: float
- 输出：产值 - float
预测数据没有输出部分，其他与预测一样。时间跨度为2016年9月1日 - 2016年11月30日
数据来自于真实的工业场景，发布者已对其进行了脱敏处理。
训练与预测都各自包含46组数据文件，每组数据代表不同数据源，组之间的温度与湿度信息一样而输出不同.
本次实验仅对其中一组数据进行LSTM时间序列预测，其他数据同。
'''
# 加载数据文件——数据采集区1
columns = ['YEAR','MONTH','DAY','TEMP_HIG','TEMP_COL','AVG_TEMP','AVG_WET','DATA_COL']
data = pd.read_csv('../input/industry/industry_timeseries/timeseries_train_data/1.csv', 
                      names=columns)
# 展示前五个数据结果
data.head()

,YEAR,MONTH,DAY,TEMP_HIG,TEMP_COL,AVG_TEMP,AVG_WET,DATA_COL
0,2015,2,1,1.9,-0.4,0.7875,75.000,907.177044
1,2015,2,2,6.2,-3.9,1.7625,77.250,747.835779
2,2015,2,3,7.8,2.0,4.2375,72.750,740.097015
3,2015,2,4,8.5,-1.2,3.0375,65.875,760.081199
4,2015,2,5,7.9,-3.6,1.8625,55.375,676.920858


In [10]:
# 查看数据采集区1的数据
plt.figure(figsize=(24,8))
for i in range(8):
    plt.subplot(8, 1, i+1)
    plt.plot(data.values[:, i])
    plt.title(columns[i], y=0.5, loc='right')
plt.show() 

<Figure size 1728x576 with 8 Axes>

In [11]:
# 观察到DATAC_COL数据的波动，随着季节和年月的变化，具有一定的规律性，称之为季节性；
# 针对该类型的数据，会有一系列的模型试用，本次结果仅用LSTM模型进行数据预测。时间原因
#，不进行不同模型结果比较。

In [12]:
# 将时间序列问题，转换为有监督的问题。
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [13]:

# 将数据归一化到0-1之间,无量纲化，即归一化处理，试用sklearn库中的 MinMaxScaler方法。
# scaler = MinMaxScaler(feature_range=(0,1))
# scaled_data = scaler.fit_transform(example[['DATA_COL','TEMP_HIG','TEMP_COL','AVG_TEMP','AVG_WET']].values)

from sklearn import preprocessing 
scaler = preprocessing.MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(data[['DATA_COL','TEMP_HIG','TEMP_COL','AVG_TEMP','AVG_WET']].values)
# scaled_data = data[['DATA_COL','TEMP_HIG','TEMP_COL','AVG_TEMP','AVG_WET']].values
print('归一化后的数据维度：',scaled_data.shape)
scaled_data[0:5][:]

归一化后的数据维度： (578, 5)


array([[0.34896842, 0.04986877, 0.26582278, 0.2126935 , 0.59916493],
       [0.25282935, 0.16272966, 0.17721519, 0.23684211, 0.63674322],
       [0.24816014, 0.20472441, 0.32658228, 0.29814241, 0.56158664],
       [0.26021767, 0.22309711, 0.24556962, 0.26842105, 0.44676409],
       [0.2100426 , 0.20734908, 0.18481013, 0.23931889, 0.27139875]])

In [14]:
# 将时序数据转换为监督问题数据，完成数据转换。
reframed = series_to_supervised(scaled_data, 1, 1)

#删除无用的label数据，当存在数据NAN时候，将该数据删除。
reframed.drop(reframed.columns[[6,7,8,9]], axis=1, inplace=True)
redf = reframed

# print(redf.info())
print('=========================================================')
print('有监督时间序列数据')
print('第一栏：当天产值；\n第二到五栏：四个影响因素；\n第六栏：明天产值')
print('=========================================================')
redf.head()


有监督时间序列数据
第一栏：当天产值；
第二到五栏：四个影响因素；
第六栏：明天产值


,var1(t-1),var2(t-1),var3(t-1),var4(t-1),var5(t-1),var1(t)
1,0.348968,0.049869,0.265823,0.212693,0.599165,0.252829
2,0.252829,0.162730,0.177215,0.236842,0.636743,0.248160
3,0.248160,0.204724,0.326582,0.298142,0.561587,0.260218
4,0.260218,0.223097,0.245570,0.268421,0.446764,0.210043
5,0.210043,0.207349,0.184810,0.239319,0.271399,0.263193


In [15]:
# 数据集划分,选取前400天的数据作为训练集,用于训练LSTM模型；
# 将中间150天作为验证集，用于观测训练过程中模型是否出现过拟合；
# 其余的作为测试集，验证模型的测试结果。
train_days = 400
valid_days = 150

values = redf.values

# 加载训练集、验证集、测试集的数据
train = values[:train_days, :]
valid = values[train_days:train_days+valid_days, :]
test = values[train_days+valid_days:, :]

# 训练集和测试集的训练样本X及预测值y。
# X为样本，y为标签。
train_X, train_y = train[:, :-1], train[:, -1]
valid_X, valid_y = valid[:, :-1], valid[:, -1]
test_X, test_y = test[:, :-1], test[:, -1]
# 训练集的X中包含四百个样本，每个样本包含产量和四个因素共五个数值。
train_X.shape, train_y.shape

((400, 5), (400,))

In [16]:
train_X[0], train_y[0]

(array([0.34896842, 0.04986877, 0.26582278, 0.2126935 , 0.59916493]),
 0.2528293521401861)

In [17]:
 
# 将数据集重构为符合LSTM要求的数据格式,即 [样本，时间步，特征].
# 在原数据样本上，增加一个维度。使数据格式满足模型的输入、输出格式。
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
valid_X = valid_X.reshape((valid_X.shape[0], 1, valid_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
print(train_X.shape, train_y.shape, valid_X.shape, valid_y.shape, test_X.shape, test_y.shape)  

train_y = train_y.reshape((train_y.shape[0], 1, 1))
valid_y = valid_y.reshape((valid_y.shape[0], 1, 1))
test_y = test_y.reshape((test_y.shape[0], 1, 1))

print(train_X.shape, train_y.shape, valid_X.shape, valid_y.shape, test_X.shape, test_y.shape)

(400, 1, 5) (400,) (150, 1, 5) (150,) (27, 1, 5) (27,)
(400, 1, 5) (400, 1, 1) (150, 1, 5) (150, 1, 1) (27, 1, 5) (27, 1, 1)


In [18]:

# 使用Keras生成LSTM模型，模型包含一个LSTM模块和一个输出。
model2 = Sequential()
# LSTM中，激活函数使用relu
model2.add(LSTM(50, activation='relu',input_shape=(train_X.shape[1], train_X.shape[2]), return_sequences=True))
# 输出结果的激活函数使用线性函数，不做非线性处理。
model2.add(Dense(1, activation='linear'))
# 使用的损失函数为MSE，最小误差方差损失，使用Adam作为优化方法。
model2.compile(loss='mean_squared_error', optimizer='adam') 
# 显示模型的结构
model2.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 1, 50)             11200     
_________________________________________________________________
dense_1 (Dense)              (None, 1, 1)              51        
Total params: 11,251
Trainable params: 11,251
Non-trainable params: 0
_________________________________________________________________


In [19]:
# from keras.utils import plot_model
# plot_model(model2, 'lstm.png')
from PIL import Image
plt.imshow(np.array(Image.open('lstm.png')))
plt.axis('off')
plt.show()

<Figure size 432x288 with 1 Axes>

In [20]:
# 训练模型
# LSTM 是进行时间序列的数据生成预测结果的一个常用深度学习模型，常用于自然语言处理。
# 在时间序列分析中也发挥着重要作用。
LSTM = model2.fit(train_X, train_y, epochs=100, batch_size=32, validation_data=(valid_X, valid_y), verbose=2, shuffle=False)


Train on 400 samples, validate on 150 samples
Epoch 1/100
 - 1s - loss: 0.0287 - val_loss: 0.0426
Epoch 2/100
 - 0s - loss: 0.0219 - val_loss: 0.0395
Epoch 3/100
 - 0s - loss: 0.0197 - val_loss: 0.0386
Epoch 4/100
 - 0s - loss: 0.0190 - val_loss: 0.0377
Epoch 5/100
 - 0s - loss: 0.0183 - val_loss: 0.0366
Epoch 6/100
 - 0s - loss: 0.0177 - val_loss: 0.0356
Epoch 7/100
 - 0s - loss: 0.0170 - val_loss: 0.0346
Epoch 8/100
 - 0s - loss: 0.0164 - val_loss: 0.0336
Epoch 9/100
 - 0s - loss: 0.0158 - val_loss: 0.0326
Epoch 10/100
 - 0s - loss: 0.0152 - val_loss: 0.0316
Epoch 11/100
 - 0s - loss: 0.0146 - val_loss: 0.0307
Epoch 12/100
 - 0s - loss: 0.0140 - val_loss: 0.0297
Epoch 13/100
 - 0s - loss: 0.0134 - val_loss: 0.0287
Epoch 14/100
 - 0s - loss: 0.0128 - val_loss: 0.0278
Epoch 15/100
 - 0s - loss: 0.0123 - val_loss: 0.0268
Epoch 16/100
 - 0s - loss: 0.0117 - val_loss: 0.0257
Epoch 17/100
 - 0s - loss: 0.0112 - val_loss: 0.0247
Epoch 18/100
 - 0s - loss: 0.0106 - val_loss: 0.0236
Epoch 19/

In [21]:
# 存储训练结果
model2.save('lstm.h5')

In [22]:
# plot history
plt.plot(LSTM.history['loss'], label='train')
plt.plot(LSTM.history['val_loss'], label='valid')
plt.legend()
plt.title('Change of Loss During Training')
plt.show()

<Figure size 432x288 with 1 Axes>

In [23]:
# 训练过程中，模型未出现过拟合现象，且损失最后收敛。

In [24]:
# 将训练数据作为输入，获取模型预测的输出结果。
train_predict = model2.predict(train_X)
# 将验证集数据作为输入，获取模型预测的输出结果。
valid_predict = model2.predict(valid_X)
# 将测试集数据作为输入，获取模型预测的输出结果。
test_predict = model2.predict(test_X)

train_predict.shape, test_predict[0:5]

((400, 1, 1), array([[[0.2793009 ]],
 
        [[0.27093697]],
 
        [[0.31213573]],
 
        [[0.2897593 ]],
 
        [[0.24541068]]], dtype=float32))

In [25]:
# 调整数据的维度，便于进行可视化呈现。
train_predict = train_predict.reshape([train_predict.shape[0],1])
valid_predict = valid_predict.reshape([valid_predict.shape[0],1])
test_predict = test_predict.reshape([test_predict.shape[0],1])
train_predict.shape

(400, 1)

In [26]:
plt.figure(figsize=(24,8))
plt.plot(values[:, -1], c='b')
plt.plot([x for x in train_predict], c='g')
plt.plot([None for _ in train_predict] + [x for x in valid_predict], c='y')
plt.plot([None for _ in train_predict] + [None for _ in valid_predict] + [x for x in test_predict], c='r')
plt.legend(['Original data', 'predicted trained data','predicted valid data', 'predicted test data'])
plt.title('Time Series Analysis')
plt.show()

<Figure size 1728x576 with 1 Axes>

In [27]:
plt.figure(figsize=(24,8))
plt.subplot(3,1,1)
plt.plot(values[:, -1], c='b')
plt.plot([x for x in train_predict], c='g')
plt.legend(['Original data', 'predicted trained data'])
plt.subplot(3,1,2)
plt.plot(values[:, -1], c='b')
plt.plot([None for _ in train_predict] + [x for x in valid_predict], c='y')
plt.legend(['Original data', 'predicted valid data'])
plt.subplot(3,1,3)
plt.plot(values[:, -1], c='b')
plt.plot([None for _ in train_predict] + [None for _ in valid_predict] + [x for x in test_predict], c='r')
plt.legend(['Original data', 'predicted test data'])
plt.show()

<Figure size 1728x576 with 3 Axes>

In [28]:
# 时间原因，只给出模型的结果，结果的误差分析未进行。
# 本次实验主要目的，是为了验证LSTM在真实场景下的工业数据是否具有良好效果，以及实现模型搭建。